In [1736]:
import numpy as np
from PIL import Image, ImageDraw, ImageColor

In [1737]:
class RotationTranslationMatrix:
    def __init__(self, m) -> None:
        assert m.shape[0] == m.shape[1]
        self.m = m

In [1738]:
class Vector:
    def __init__(self, v):
        self.v = v

    def __eq__(self, __o):
        return np.allclose(self.v, __o.v)

    def __add__(self, __o):
        return Vector(self.v + __o.v)

    def __sub__(self, __o):
        return Vector(self.v - __o.v)

    def __mul__(self, __o):
        return Vector(self.v * __o)

    def norm(self):
        return np.linalg.norm(self.v)

    def unit(self):
        return self.v / self.norm()
    
    def __repr__(self) -> str:
        return f"Vector {self.v}"

In [1739]:
class Point:
    def __init__(self, *xyz, nparr=None):
        self.xyz = np.array(xyz) if xyz else nparr

    def as_affine(self):
        self.xyz = self.xyz[:-1] / self.xyz[-1]

    def as_homogenious(self):
        self.xyz = np.append(self.xyz, 1) # return new?

    def to_vec(self):
        return Vector(self.xyz)

    def __eq__(self, __o):
        return np.allclose(self.xyz, __o.xyz)

    def __add__(self, __o: Vector):
        return Point(nparr=self.xyz + __o.v)

    def __sub__(self, __o):
        return Vector(self.xyz - __o.xyz)

    # def __mul__(self, __o):
    #     return Point(self.xyz * __o)

    # def __matmul__(self, m):
    #     return Vector(self.as_homogenious() @ m).to_affine()

    def __repr__(self) -> str:
        return f"Point {self.xyz}"

In [1740]:
class Edge:
    def __init__(
        self,
        a,
        b,
    ):
        self.a = a
        self.b = b
        self.v = b - a

    def __eq__(self, __o):
        return (
            self.a == __o.a and self.b == __o.b or self.a == __o.b and self.b == __o.a
        )

    def normal(self, viewpoint):
        v_a = self.a - viewpoint
        v_b = self.b - self.a
        proj = np.dot(v_a, v_b) * v_b / v_b.norm() ** 2
        n = (v_a - proj).unit()
        return n

    def __repr__(self) -> str:
        return f"Edge <{self.a} {self.b}>"

In [1741]:
# TODO
# 1) REDEFINE INITIALIZATION ✅
# 2) DEFINE ROTATION (CLOCKWISE / COUNTERCLOCKWISE)
# 3) DEFINE NORMALS FOR EDGES
# 4) DELAUNEY TRIANGUALTION
# 5) SPLIT EDGES AND VERTICES


class Polygon:
    def __init__(self, **kwargs) -> None:

        self.vertices = None
        self.edges = None
        self.rotation = None
        self.L = None
        self.convex_type = None
        self.simple_type = None
        self.triangluation = None
        self.split_edges = None
        self.split_vertices = None
        self.color = "black"
        self.__dict__.update(kwargs)

        assert self.vertices or self.edges, "Polygon: vertices and edges are empty"

        if self.edges is None: 
            self._edges()

        if self.vertices is None: 
            self._vertices()

        if self.L is None: 
            self.L = len(self.vertices)

        if self.convex_type is None:
            self._convex_type()

        if self.simple_type is None:
            self._simple_type()

        if self.split_edges is None:
            self.split_edges = []
            self.split_vertices = []
            self._split_edges_and_vertices()

        if self.rotation is None:
            self._rotation()

        if self.triangluation is None:
            self.triangluation = self._triangluation()

        self.color_rgb = ImageColor.getrgb(self.color)

    def _edges(self):
        self.edges = [
            Edge(p1, p2)
            for p1, p2 in zip(
                self.vertices, self.vertices[1:] + [self.vertices[0]]
            )
        ]

    def _vertices(self):
        self.vertices = [edge[0] for edge in self.edges]

    def _convex_type(self):
        flag = None
        for i in range(self.L):
            for j in range(i + 1, i + self.L - 1):
                cl_type = classifyEdgePoint(self.edges[j % self.L], self.vertices[i])
                if flag is None:
                    flag = cl_type
                elif flag != cl_type:
                    self.convex_type = "CONCAVE"
                    return
        self.convex_type = "CONVEX"

    def _simple_type(self):
        flag = "SIMPLE"
        intersection_pairs = {}
        for i in range(self.L - 1):
            for j in range(i + 2, self.L):
                if i == (j + 1) % self.L:
                    continue
                vi, vj = self.edges[i], self.edges[j]
                cross_type, ti = crossEdges(vi, vj)
                if cross_type == "SKEW_CROSS":
                    intersection_point = vi.a + (vi.b - vi.a) * ti
                    tj = (intersection_point - vj.a).norm() / vj.v.norm()
                    intersection_pairs[(i, j)] = (intersection_point, ti, tj)
                    flag = "SELF_INTERSECTED"
        self.simple_type = flag
        self._intersection_pairs = intersection_pairs

    def _rotation(self):
        signed_area = self.signed_area()
        if signed_area > 0:
            self.rotation = "CCW"
        else:
            self.rotation = "CW"

    def _split_edges_and_vertices(self):
        # Step 1: Collect intersection points per edge
        splits = {}  # edge_idx -> [(t_param, point), ...]
        for (i, j), (point, ti, tj) in self._intersection_pairs.items():
            for idx, t in ((i, ti), (j, tj)):
                splits.setdefault(idx, []).append((t, point))
        
        # Sort splits by parameter t for each edge
        for split_list in splits.values():
            split_list.sort(key=lambda x: x[0])
        
        # Build new edges from sorted splits
        split_edges = {}
        for idx, split_list in splits.items():
            edge = self.edges[idx]
            segments = [edge.a] + [pt for _, pt in split_list] + [edge.b]
            split_edges[idx] = [Edge(segments[i], segments[i+1]) for i in range(len(segments)-1)]

        stack_edges = [[]]
        stack_vetrices = [[]]
        for i in range(self.L): 
            if i not in split_edges:
                stack_edges[-1].append(self.edges[i])
                stack_vetrices[-1].append(self.vertices[i])
                if len(stack_edges[-1]) > 1 and \
                    stack_edges[-1][0].a == stack_edges[-1][-1].b:
                    self.split_edges.append(stack_edges.pop())
                    self.split_vertices.append(stack_vetrices.pop())
            else:
                for edge in split_edges[i]:
                    stack_edges[-1].append(edge)
                    stack_vetrices[-1].append(edge.a)
                    if len(stack_edges[-1]) > 1 and \
                        stack_edges[-1][0].a == stack_edges[-1][-1].b:
                        self.split_edges.append(stack_edges.pop())
                        self.split_vertices.append(stack_vetrices.pop())
                    else:
                        stack_edges.append([])
                        stack_vetrices.append([])
                if stack_edges:
                    del stack_edges[-1]
                    del stack_vetrices[-1]
    

    def signed_area(self):
        for edges in self.split_edges:
            print(0.5 * sum(np.cross(edge.a.xyz, edge.b.xyz) for edge in edges))
        signed_area = 0.5 * sum(np.cross(edge.a.xyz, edge.b.xyz) for edges in self.split_edges for edge in edges)
        return signed_area
    

    def EO(self, p):
        intersection_sum = 0
        for edge in self.edges:
            match classifyEdgePoint(edge, p):
                case "TOUCHING":
                    return "INSIDE"
                case "CROSS_LEFT":
                    intersection_sum += 1
                case "CROSS_RIGHT":
                    intersection_sum += 1
        if intersection_sum % 2 == 1:
            return "INSIDE"
        return "OUTSIDE"

    def NZW(self, p):
        winding_number = 0
        for edge in self.edges:
            match classifyEdgePoint(edge, p):
                case "TOUCHING":
                    return "INSIDE"
                case "CROSS_LEFT":
                    winding_number += 1
                case "CROSS_RIGHT":
                    winding_number -= 1
        if winding_number == 0:
            return "OUTSIDE"
        return "INSIDE"

    def _triangluation(self):
        return 1
    #     # return adjacency matrix?
    #     all_points = self.vertices
    #     for edge in self.edges:
    #         all_points.extend(edge.intersection_points())
    #     all_points = sorted(all_points, lambda x: x.xyz[0])
    #     triangualtion = Polygon(*all_points[:3], color=tricolor)
    #     for point in all_points[3:]:
    #         for edge in triangualtion:
    #             view_dir = Vector(edge[0] - point)
    #         poly = Polygon(
    #             edges=triangualtion.edges+[Edge(v, point) for v in triangualtion.vertices],
    #             color=tricolor
    #         )

In [1742]:
def edge_visible(): ...

In [1743]:
def check_edge_visibility(face, view_direction=np.array([0, 0, 1000])):
    v1, v2, v3, *_ = face
    normal = np.cross(np.array([*(v3 - v1)]), np.array([*(v2 - v1)]))
    dot_product = np.dot(normal, view_direction)
    return dot_product <= 0

In [1744]:
def draw_polygon(polygon, draw):
    for edge in polygon.edges:
        draw.point(pixelEdge(edge), fill=polygon.color)

In [1745]:
def pixelLine(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    ix = 1 if dx > 0 else -1
    iy = 1 if dy > 0 else -1
    dx *= ix
    dy *= iy
    x = x1
    y = y1
    pixels = []
    if dx < dy:
        E = 2 * dx - dy
        for _ in range(dy + 1):
            pixels.append((x, y))
            if E >= 0:
                x += ix
                E -= 2 * dy
            y += iy
            E += 2 * dx
    else:
        E = 2 * dy - dx
        for _ in range(dx + 1):
            pixels.append((x, y))
            if E >= 0:
                y += iy
                E -= 2 * dx
            x += ix
            E += 2 * dy
    return pixels


def pixelEdge(e):
    x1, y1, x2, y2 = (
        round(e.a.xyz[0]),
        round(e.a.xyz[1]),
        round(e.b.xyz[0]),
        round(e.b.xyz[1]),
    )
    return pixelLine(x1, y1, x2, y2)

In [1746]:
def classify(x1, y1, x2, y2, x, y):
    ax = x2 - x1
    ay = y2 - y1
    bx = x - x1
    by = y - y1
    s = ax * by - ay * bx
    if s > 0:
        return "LEFT"
    if s < 0:
        return "RIGHT"
    if ax * bx < 0 or ay * by < 0:
        return "BEHIND"
    if ax**2 + ay**2 < bx**2 + by**2:
        return "INFRONT"
    if x == x1 and y == y1:
        return "ORIGIN"
    if x == x2 and y == y2:
        return "DESTINATION"
    return "BETWEEN"


def classifyEdgePoint(e, p):
    return classify(*e.a.xyz, *e.b.xyz, *p.xyz)

In [1747]:
def intersect(ax, ay, bx, by, cx, cy, dx, dy):
    nx = dy - cy
    ny = cx - dx
    denom = nx * (bx - ax) + ny * (by - ay)
    if denom == 0:
        type = classify(cx, cy, dx, dy, ax, ay)
        if type == "LEFT" or type == "RIGHT":
            return "PARALLEL", None
        else:
            return "SAME", None
    num = nx * (ax - cx) + ny * (ay - cy)
    t = -num / denom
    return "SKEW", t


def intersectEdges(e1, e2):
    return intersect(*e1.a.xyz, *e1.b.xyz, *e2.a.xyz, *e2.b.xyz)

In [1748]:
def cross(ax, ay, bx, by, cx, cy, dx, dy):
    type, tab = intersect(ax, ay, bx, by, cx, cy, dx, dy)
    if type == "SAME" or type == "PARALLEL" or tab is None:
        return type, None
    if tab < 0 or tab > 1:
        return "SKEW_NO_CROSS", None
    _, tcd = intersect(cx, cy, dx, dy, ax, ay, bx, by)
    if tcd is not None and (tcd < 0 or tcd > 1):
        return "SKEW_NO_CROSS", None
    return "SKEW_CROSS", tab


def crossEdges(e1, e2):
    return cross(*e1.a.xyz, *e1.b.xyz, *e2.a.xyz, *e2.b.xyz)

In [1749]:
img = Image.new("RGB", (200, 200), "white")
draw = ImageDraw.Draw(img)

In [1750]:
p0 = Polygon(
    vertices=[
        Point(150, 80),
        Point(10, 60),
        Point(50, 100),
        Point(100, 20),
        Point(120, 160),
        Point(190, 100),
        Point(180, 190),
    ],
    color="blue"
)

-1022.9508196721315
1005.2424863387978
1140.0
-2922.291666666666


In [1751]:
p0.convex_type

'CONCAVE'

In [1752]:
p0.simple_type

'SELF_INTERSECTED'

In [1753]:
p0.split_edges

[[Edge <Point [69.67213115 68.52459016] Point [10 60]>,
  Edge <Point [10 60] Point [ 50 100]>,
  Edge <Point [ 50 100] Point [69.67213115 68.52459016]>],
 [Edge <Point [107.70833333  73.95833333] Point [69.67213115 68.52459016]>,
  Edge <Point [69.67213115 68.52459016] Point [100  20]>,
  Edge <Point [100  20] Point [107.70833333  73.95833333]>],
 [Edge <Point [162. 124.] Point [190 100]>,
  Edge <Point [190 100] Point [180 190]>,
  Edge <Point [180 190] Point [162. 124.]>],
 [Edge <Point [150  80] Point [107.70833333  73.95833333]>,
  Edge <Point [107.70833333  73.95833333] Point [120 160]>,
  Edge <Point [120 160] Point [162. 124.]>,
  Edge <Point [162. 124.] Point [150  80]>]]

In [1754]:
p0.signed_area()

-1022.9508196721315
1005.2424863387978
1140.0
-2922.291666666666


-1800.0

In [1755]:
p0.rotation

'CW'

In [1756]:
draw_polygon(p0, draw)

In [1757]:
p1 = Polygon(
    vertices=[
        Point(10, 10),
        Point(40, 100),
        Point(100, 150),
        Point(50, 180),
        Point(70, 10),
    ],
    color="green",
)

1516.5178571428569
-3716.517857142857


In [1758]:
p1.convex_type

'CONCAVE'

In [1759]:
p1.simple_type

'SELF_INTERSECTED'

In [1760]:
p1.split_edges

[[Edge <Point [ 57.67857143 114.73214286] Point [100 150]>,
  Edge <Point [100 150] Point [ 50 180]>,
  Edge <Point [ 50 180] Point [ 57.67857143 114.73214286]>],
 [Edge <Point [10 10] Point [ 40 100]>,
  Edge <Point [ 40 100] Point [ 57.67857143 114.73214286]>,
  Edge <Point [ 57.67857143 114.73214286] Point [70 10]>,
  Edge <Point [70 10] Point [10 10]>]]

In [1761]:
# draw_polygon(p1, draw)

In [1762]:
p2 = Polygon(
    vertices=[
        Point(20, 10),
        Point(50, 10),
        Point(80, 20),
        Point(90, 50),
        Point(70, 80),
        Point(40, 90),
        Point(10, 60),
    ],
    color="yellow",
)

4650.0


In [1763]:
p2.convex_type

'CONVEX'

In [1764]:
p2.simple_type

'SIMPLE'

In [1765]:
p2.split_edges

[[Edge <Point [20 10] Point [50 10]>,
  Edge <Point [50 10] Point [80 20]>,
  Edge <Point [80 20] Point [90 50]>,
  Edge <Point [90 50] Point [70 80]>,
  Edge <Point [70 80] Point [40 90]>,
  Edge <Point [40 90] Point [10 60]>,
  Edge <Point [10 60] Point [20 10]>]]

In [1766]:
# draw_polygon(p2, draw)
# img.show()

In [1767]:
triangle = Polygon(
    vertices=[
        Point(34, 80), 
        Point(57, 1), 
        Point(170, 80),
        ], 
        color="yellow"  # A B  # B C  # C A
)

5372.0


In [1768]:
def paint_triangle():
    edges = [pixelEdge(edge) for edge in triangle.edges]
    ABC = edges[0] + edges[1]
    i = 0
    AC = edges[2][::-1]
    j = 0
    A = AC[0]
    C = AC[-1]
    dxAC = abs(A[0] - C[0])
    dyAC = abs(A[1] - C[1])
    k = 0 if dxAC > dyAC else 1
    pixels = []
    while ABC[i] != C:
        line = pixelLine(*ABC[i], *AC[j])
        pixels.extend(line)
        i += 1
        if AC[j][k] != ABC[i][k]:
            j += 1
    return pixels

In [1769]:
# draw.point(paint_triangle(), fill="red")
img.show()